In [1]:
import os
import sys
import json
from datetime import datetime

class Colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

In [ ]:
def check_dependencies():
    """检查依赖包"""
    print(f"\n{Colors.OKBLUE}[1/5] 检查依赖包...{Colors.ENDC}")
    
    required = ['requests', 'pandas', 'numpy', 'tabulate']
    missing = []
    
    for package in required:
        try:
            __import__(package)
            print(f"  ✓ {package}")
        except ImportError:
            missing.append(package)
            print(f"  ✗ {package} {Colors.FAIL}(未安装){Colors.ENDC}")
    
    if missing:
        print(f"\n{Colors.FAIL}缺少依赖包！请运行：{Colors.ENDC}")
        print(f"  pip install {' '.join(missing)}")
        return False
    
    print(f"{Colors.OKGREEN}依赖检查通过！{Colors.ENDC}")
    return True


def check_config():
    """检查配置文件"""
    print(f"\n{Colors.OKBLUE}[2/5] 检查配置文件...{Colors.ENDC}")
    
    try:
        from config import API_CONFIG, TRADING_PAIRS, STRATEGY_CONFIG, DATA_CONFIG
        
        print(f"  ✓ API配置")
        print(f"  ✓ 交易对: {', '.join(TRADING_PAIRS)}")
        print(f"  ✓ 策略参数")
        
        # 显示关键配置
        print(f"\n  关键参数:")
        print(f"    - 再平衡间隔: {STRATEGY_CONFIG['rebalance_interval']} 分钟")
        print(f"    - 最小数据点: {STRATEGY_CONFIG['min_data_points']} 个")
        print(f"    - 最小交易额: ${STRATEGY_CONFIG['min_position_value']}")
        print(f"    - 最大单仓位: {STRATEGY_CONFIG['max_position_pct']*100}%")
        
        print(f"{Colors.OKGREEN}配置检查通过！{Colors.ENDC}")
        return True
        
    except ImportError as e:
        print(f"{Colors.FAIL}配置文件缺失: {e}{Colors.ENDC}")
        return False
    except Exception as e:
        print(f"{Colors.FAIL}配置文件错误: {e}{Colors.ENDC}")
        return False


def check_historical_data():
    """检查历史数据"""
    print(f"\n{Colors.OKBLUE}[3/5] 检查历史数据...{Colors.ENDC}")
    
    data_file = 'price_history.json'
    
    if os.path.exists(data_file):
        try:
            with open(data_file, 'r') as f:
                data = json.load(f)
            
            print(f"  ✓ 找到历史数据文件")
            
            for pair, records in data.items():
                print(f"    - {pair}: {len(records)} 个数据点")
            
            print(f"{Colors.OKGREEN}历史数据可用！{Colors.ENDC}")
            return True
            
        except Exception as e:
            print(f"{Colors.WARNING}历史数据文件损坏: {e}{Colors.ENDC}")
            return False
    else:
        print(f"{Colors.WARNING}未找到历史数据，将从头开始收集{Colors.ENDC}")
        return True


def test_api_connection():
    """测试API连接"""
    print(f"\n{Colors.OKBLUE}[4/5] 测试API连接...{Colors.ENDC}")
    
    try:
        from alpha_live_trading import RoostooAPIClient
        from config import API_CONFIG
        
        api = RoostooAPIClient(**API_CONFIG)
        
        # 测试获取余额
        print("  测试获取账户余额...")
        balance = api.get_balance()
        
        if balance and balance.get('Success'):
            wallet = balance.get('Wallet', {})
            usd = wallet.get('USD', {}).get('Free', 0)
            print(f"  ✓ API连接成功")
            print(f"  ✓ 账户余额: ${float(usd):,.2f} USD")
        else:
            print(f"{Colors.FAIL}  ✗ API返回错误{Colors.ENDC}")
            return False
        
        # 测试获取行情
        print("  测试获取行情数据...")
        ticker = api.get_ticker()
        
        if ticker and ticker.get('Success'):
            print(f"  ✓ 行情数据获取成功")
        else:
            print(f"{Colors.WARNING}  ! 行情数据获取失败（可以继续）{Colors.ENDC}")
        
        print(f"{Colors.OKGREEN}API连接测试通过！{Colors.ENDC}")
        return True
        
    except Exception as e:
        print(f"{Colors.FAIL}API连接失败: {e}{Colors.ENDC}")
        return False

def confirm_start():
    """确认启动"""
    print(f"\n{Colors.BOLD}准备启动交易系统...{Colors.ENDC}")
    
    response = input(f"\n{Colors.WARNING}确认启动？(输入 YES 继续): {Colors.ENDC}")
    
    if response.strip().upper() == 'YES':
        return True
    else:
        print(f"{Colors.FAIL}启动已取消{Colors.ENDC}")
        return False


In [3]:
checks = [
    check_dependencies,
    check_config,
    check_historical_data,
    test_api_connection,
]

for check in checks:
    if not check():
        print(f"\n{Colors.FAIL}启动检查失败，请修复上述问题后重试{Colors.ENDC}")
        sys.exit(1)


[1/5] 检查依赖包...
  ✓ requests
  ✓ pandas
  ✓ numpy
  ✓ tabulate
依赖检查通过！

[2/5] 检查配置文件...
  ✓ API配置
  ✓ 交易对: BTC/USD, ETH/USD, BNB/USD, XRP/USD, DOGE/USD, SOL/USD, ARB/USD
  ✓ 策略参数

  关键参数:
    - 再平衡间隔: 60 分钟
    - 最小数据点: 100 个
    - 最小交易额: $10.0
    - 最大单仓位: 30.0%
配置检查通过！

[3/5] 检查历史数据...
未找到历史数据，将从头开始收集

[4/5] 测试API连接...
  测试获取账户余额...
  ✓ API连接成功
  ✓ 账户余额: $0.00 USD
  测试获取行情数据...
  ✓ 行情数据获取成功
API连接测试通过！


In [ ]:
from alpha_live_trading import AlphaLiveTrading, RoostooAPIClient
from config import API_CONFIG, TRADING_PAIRS, STRATEGY_CONFIG, DATA_CONFIG

# 初始化
api_client = RoostooAPIClient(**API_CONFIG)

live_trading = AlphaLiveTrading(
    api_client=api_client,
    trading_pairs=TRADING_PAIRS,
    **STRATEGY_CONFIG
)

# 启动
live_trading.run_forever(
    data_collection_interval=DATA_CONFIG['collection_interval']
)

2025-11-10 14:38:47,332 - alpha_live_trading - INFO - ======================================================================
2025-11-10 14:38:47,332 - alpha_live_trading - INFO - 启动Alpha实盘交易系统
2025-11-10 14:38:47,333 - alpha_live_trading - INFO - 交易对: ['BTC/USD', 'ETH/USD', 'BNB/USD', 'XRP/USD', 'DOGE/USD', 'SOL/USD', 'ARB/USD']
2025-11-10 14:38:47,334 - alpha_live_trading - INFO - 数据收集间隔: 60秒
2025-11-10 14:38:47,334 - alpha_live_trading - INFO - 再平衡间隔: 60分钟
2025-11-10 14:38:47,336 - alpha_live_trading - INFO - 最少数据点: 100
2025-11-10 14:38:47,336 - alpha_live_trading - INFO - ======================================================================
2025-11-10 14:38:47,337 - alpha_live_trading - INFO - 
[迭代 1] 2025-11-10 14:38:47
2025-11-10 14:38:47,479 - alpha_live_trading - INFO - [数据收集] 获取价格: {'BTC/USD': 106120.58, 'ETH/USD': 3603.17, 'BNB/USD': 1005.54, 'XRP/USD': 2.4546, 'DOGE/USD': 0.18044, 'SOL/USD': 166.77, 'ARB/USD': 0.3005}
2025-11-10 14:38:47,480 - alpha_live_trading - INFO - [数据